In [79]:
import pandas as pd
import  numpy as np
from sklearn.metrics import confusion_matrix
import category_encoders as ce
from sklearn import preprocessing
from  sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,StackingClassifier,VotingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import seaborn as sns

In [80]:
Train_data =pd.read_csv("Data/traindata.csv")
X_test=pd.read_csv("Data/testdata.csv")

In [81]:
Train_data.head(5)

,ID,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level,Class
0,0,0,tcp,http,SF,295,2584,0,0,0,...,1.00,0.00,0.01,0.03,0.0,0.0,0.00,0.01,21,0
1,1,0,tcp,http,SF,312,839,0,0,0,...,1.00,0.00,0.00,0.02,0.0,0.0,0.01,0.01,21,0
2,2,0,tcp,ftp,SF,1225,2451,0,0,0,...,0.45,0.02,0.00,0.00,0.0,0.0,0.00,0.00,11,1
3,3,0,tcp,http,SF,293,296,0,0,0,...,1.00,0.00,0.00,0.02,0.0,0.0,0.00,0.02,21,0
4,4,0,tcp,X11,REJ,0,0,0,0,0,...,0.01,0.03,0.01,0.00,0.0,0.0,0.01,1.00,11,0


In [82]:
Train_data.shape

(23002, 44)

In [83]:
Train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23002 entries, 0 to 23001
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           23002 non-null  int64  
 1   duration                     23002 non-null  int64  
 2   protocol_type                23002 non-null  object 
 3   service                      23002 non-null  object 
 4   flag                         23002 non-null  object 
 5   src_bytes                    23002 non-null  int64  
 6   dst_bytes                    23002 non-null  int64  
 7   land                         23002 non-null  int64  
 8   wrong_fragment               23002 non-null  int64  
 9   urgent                       23002 non-null  int64  
 10  hot                          23002 non-null  int64  
 11  num_failed_logins            23002 non-null  int64  
 12  logged_in                    23002 non-null  int64  
 13  num_compromised 

In [84]:
Train_data.duplicated().sum()

0

In [85]:
Train_data["Class"].value_counts()

Class
0    12001
1    11001
Name: count, dtype: int64

# Check the columns that has the same values for all rows

In [86]:
for column in Train_data.select_dtypes(include=['number']):
    variance = Train_data[column].var()
    if variance == 0:
        print(f"All values in '{column}' are the same.")
    else:
        print(f"Values in '{column}' are not the same.")

Values in 'ID' are not the same.
Values in 'duration' are not the same.
Values in 'src_bytes' are not the same.
Values in 'dst_bytes' are not the same.
Values in 'land' are not the same.
Values in 'wrong_fragment' are not the same.
Values in 'urgent' are not the same.
Values in 'hot' are not the same.
Values in 'num_failed_logins' are not the same.
Values in 'logged_in' are not the same.
Values in 'num_compromised' are not the same.
Values in 'root_shell' are not the same.
Values in 'su_attempted' are not the same.
Values in 'num_root' are not the same.
Values in 'num_file_creations' are not the same.
Values in 'num_shells' are not the same.
Values in 'num_access_files' are not the same.
All values in 'num_outbound_cmds' are the same.
All values in 'is_host_login' are the same.
Values in 'is_guest_login' are not the same.
Values in 'count' are not the same.
Values in 'srv_count' are not the same.
Values in 'serror_rate' are not the same.
Values in 'srv_serror_rate' are not the same.
Va

In [87]:
Train_data["num_outbound_cmds"].value_counts()

num_outbound_cmds
0    23002
Name: count, dtype: int64

In [88]:
Train_data["is_host_login"].value_counts()

is_host_login
0    23002
Name: count, dtype: int64

# Drop the columns that not changed for all rows

In [89]:
Train_data.drop(columns=['num_outbound_cmds', 'ID',"is_host_login"],inplace=True)

In [90]:
X_test.drop(columns=['num_outbound_cmds', 'ID',"is_host_login"],inplace=True)

In [91]:
Train_data.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level,Class
count,23002.000000,2.300200e+04,2.300200e+04,23002.000000,23002.000000,23002.000000,23002.000000,23002.000000,23002.000000,23002.000000,...,23002.000000,23002.000000,23002.000000,23002.000000,23002.000000,23002.000000,23002.000000,23002.000000,23002.000000,23002.000000
mean,307.196896,2.588955e+04,3.588139e+03,0.000087,0.024259,0.000043,0.199113,0.001174,0.386532,0.231241,...,0.511856,0.084503,0.147974,0.031739,0.293264,0.287243,0.119363,0.120072,19.467220,0.478263
std,2697.345784,2.522555e+06,9.231127e+04,0.009324,0.262794,0.006594,2.167079,0.046141,0.486965,10.778668,...,0.448993,0.189916,0.309429,0.111354,0.448771,0.449637,0.307603,0.319011,2.331899,0.499538
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000
50%,0.000000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.460000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000
75%,0.000000,2.747500e+02,4.860000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,1.000000,0.070000,0.060000,0.010000,1.000000,1.000000,0.000000,0.000000,21.000000,1.000000
max,42862.000000,3.817091e+08,5.151385e+06,1.000000,3.000000,1.000000,77.000000,4.000000,1.000000,884.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,21.000000,1.000000


In [92]:
Train_data["Class"].value_counts()

Class
0    12001
1    11001
Name: count, dtype: int64

# Label Encoding

In [93]:
encoder_protocol_type = preprocessing.LabelEncoder()

Train_data['protocol_type']= encoder_protocol_type.fit_transform(Train_data['protocol_type'])
X_test['protocol_type']= encoder_protocol_type.transform(X_test['protocol_type'])



In [94]:
X_test.drop(columns=['service'],inplace=True)
Train_data.drop(columns=['service'],inplace=True)

In [95]:

encoder_flag = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
Train_data['flag']= encoder_flag.fit_transform(Train_data['flag'])
X_test['flag']= encoder_flag.transform(X_test['flag'])



In [96]:
x_train = Train_data.drop(columns=['Class'])
y_train = Train_data['Class']

# appply RandomForestClassifier

In [97]:
model = RandomForestClassifier(min_samples_leaf= 37
                              ,max_depth=15, n_estimators=380, random_state=42,max_features=6 ,class_weight={0:1,1:99})

In [98]:
model.fit(x_train, y_train)

RandomForestClassifier(class_weight={0: 1, 1: 99}, max_depth=15, max_features=6,
                       min_samples_leaf=37, n_estimators=380, random_state=42)

In [99]:
Cat_y_pred=model.predict(X_test)

In [100]:
OUTPUT_sample=pd.DataFrame({"Class":Cat_y_pred})
OUTPUT_sample=OUTPUT_sample.rename_axis("ID")
OUTPUT_sample
OUTPUT_sample.to_csv("sampleSubmission_Model_3.csv")